In [3]:
%pip install kagglehub

  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.7 kB ? eta -:--:--
     --------------------------------- ---- 51.2/57.7 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 765.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/62.7 kB ? eta -:--:--
   ---------------------------------------  61.4/62.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 62.7/62.7 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/162.0 kB ? eta -:--:--
   ----------------------------------- ---- 143.4/162.0 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 162.0/162.0 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00
Note: you may need


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\luxma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("jrobischon/wikipedia-movie-plots")

print("Path to dataset files:", path)

df = pd.read_csv(path+"\wiki_movie_plots_deduped.csv")
df.to_csv('movies.csv', index=False)

Path to dataset files: C:\Users\luxma\.cache\kagglehub\datasets\jrobischon\wikipedia-movie-plots\versions\1


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the dataset (assuming CSV format after kagglehub download)
df = pd.read_csv("movies.csv")

# Check for plot column and preprocess
df = df.drop(columns=["Director","Cast","Wiki Page"])
df = df[df["Origin/Ethnicity"]=="American"]
df = df.drop(columns=["Origin/Ethnicity"])
df = df.dropna(subset=["Plot"]).reset_index(drop=True)
df = df.sort_values(by='Release Year', ascending=False).head(500)

# Preview dataset
print(df.head())

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words="english", max_df=0.8)
tfidf_matrix = vectorizer.fit_transform(df["Plot"])

# Recommender function
def recommend_movies(input_sentence, top_n=5):
    """
    Recommend top_n movies based on plot similarity to input_sentence.
    """
    input_vec = vectorizer.transform([input_sentence])
    # Use cosine similarity
    similarity_scores = cosine_similarity(input_vec, tfidf_matrix).flatten()
    
    # Get indices of top_n similar plots
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    
    recommendations = df.iloc[top_indices][["Title", "Release Year", "Plot"]]
    recommendations["Similarity Score"] = similarity_scores[top_indices]
    
    return recommendations

    

       Release Year                 Title                  Genre  \
17315          2017            Flatliners  horror, sci-fi, drama   
17346          2017                   LBJ       biography, drama   
17361          2017  Just Getting Started         action, comedy   
17376          2017        Phantom Thread                  drama   
17360          2017              I, Tonya       biography, drama   

                                                    Plot  
17315  A medical student, Courtney (Ellen Page), is o...  
17346  The story centers around the political upheava...  
17361  Defense lawyer Duke, who once defended the hea...  
17376  In 1954 London, renowned fashion designer Reyn...  
17360  In 1970s Portland, Oregon, three-year-old Tony...  


In [14]:
query = "I love thrilling action movies, with a comedic twist."
top_recommendations = recommend_movies(query, top_n=5)

print("Top Movie Recommendations:\n")
print(top_recommendations)

Top Movie Recommendations:

                     Title  Release Year  \
16924        Strange Magic          2015   
17058    The Perfect Match          2016   
17041        Hail, Caesar!          2016   
17302    The Good Catholic          2017   
17357  The Disaster Artist          2017   

                                                    Plot  Similarity Score  
16924  A realm divided between a land of fairies and ...          0.049546  
17058  Charlie is a playboy who's convinced that rela...          0.035614  
17041  In 1951, Eddie Mannix (Josh Brolin) is the hea...          0.026443  
17302  Three Catholic priests live in a rectory toget...          0.023802  
17357  San Francisco, 1998: 19-year-old Greg Sestero ...          0.020480  


In [7]:
import sys
print(sys.version)

3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
